In [146]:
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()
import numpy as np
import pandas as pd
import talib
import time
from datetime import datetime
import matplotlib.pyplot as plt 
import string as str
import plotly.graph_objects as go
import colorlover as cl

In [55]:
df = pd.read_csv("ind_nifty100list.csv")
stocks = df['Symbol'].unique().tolist() #Sending Output string
stocks[12] = 'BAJAJ_AUTO'
stocks[62] = 'M_M'

In [4]:
def label_change(row):
 if row['sign_change']:
  if row['sign'] == 1:
   return 'Buy' #Negative to Positive
  else:
   return 'Sell' #Positive to Negative
 return None

In [5]:
all_cases_df = pd.DataFrame()

In [112]:
def process_stock(symbol):
    try:
        stock_history = tv.get_hist(symbol=symbol, exchange='NSE', interval=Interval.in_daily, n_bars=300)
        close_values = stock_history['close']
        low_values = stock_history['low']
        macd_stock = talib.MACD(stock_history['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        ema_stock = talib.EMA(stock_history['close'], timeperiod=9)
        df = pd.DataFrame({'time': close_values.index, 'close_price': close_values.values, 'macd-signal': macd_stock[2].values, 'ema9': ema_stock.values})
        # ema_max50
        df['ema9_max'] = df['ema9'].rolling(window=50, min_periods=1).max()
        df['ema9_min'] = df['ema9'].rolling(window=50, min_periods=1).min()
        ema_max_50 = df[df['ema9'] == df['ema9_max']]  # ema-max-50days
        ema_min_50 = df[df['ema9'] == df['ema9_min']]  # ema-min-50days
        df['is_50_day_high'] = df['ema9'] == df['ema9_max']  # Boolean column
        df['is_50_day_low'] = df['ema9'] == df['ema9_min']  # Boolean column
        df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')
        df['sign'] = df['macd-signal'].apply(lambda x: 1 if x > 0 else -1)
        df['sign_change'] = df['sign'] != df['sign'].shift(1)
        df['action'] = df.apply(label_change, axis=1)
        df['stock_name'] = symbol
        buy_cases_df = df[(df['is_50_day_high'] == True) & (df['action'] == 'Buy')][['stock_name', 'time', 'close_price', 'action', 'ema9_max', 'macd-signal']]# Collecting all buy cases
        sell_cases_df = df[(df['is_50_day_low'] == True) & (df['action'] == 'Sell')][['stock_name', 'time', 'close_price', 'action', 'ema9_min', 'macd-signal']]# Collecting all sell cases
        combined_df = pd.concat([buy_cases_df, sell_cases_df])# Combine buy_cases_df and sell_cases_df
        combined_df = combined_df.sort_values(by='time', ascending=True)# Sort combined dataframe by time in descending order
        combined_df.reset_index(drop=True, inplace=True)# Reset index for clean output
        return combined_df
    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        return None
all_cases_df = pd.DataFrame()
retries = 5

for i in range(len(stocks)):
    symbol = stocks[i]
    for attempt in range(retries):
        result_df = process_stock(symbol)
        if result_df is not None:
            all_cases_df = pd.concat([all_cases_df, result_df], ignore_index=True)
            break
        else:
            print(f"Retrying {symbol}... Attempt {attempt + 1} of {retries}")
            time.sleep(1)  

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error processing ABB: 'NoneType' object is not subscriptable
Retrying ABB... Attempt 1 of 5


C:\Users\91977\AppData\Local\Temp\ipykernel_7448\59275534.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')
C:\Users\91977\AppData\Local\Temp\ipykernel_7448\59275534.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')
C:\Users\91977\AppData\Local\Temp\ipykernel_7448\59275534.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')
C:\Users\91977\AppData\Local\Temp\ipykernel_7448\59275534.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future

Error processing AMBUJACEM: 'NoneType' object is not subscriptable
Retrying AMBUJACEM... Attempt 1 of 5


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


...

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error processing ZOMATO: 'NoneType' object is not subscriptable
Retrying ZOMATO... Attempt 2 of 5


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error processing ZOMATO: 'NoneType' object is not subscriptable
Retrying ZOMATO... Attempt 3 of 5


C:\Users\91977\AppData\Local\Temp\ipykernel_7448\59275534.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')
C:\Users\91977\AppData\Local\Temp\ipykernel_7448\59275534.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')


In [113]:
all_cases_df

,stock_name,time,close_price,action,ema9_max,macd-signal,ema9_min
0,ABB,2023-08-09 09:15:00,4520.95020,Buy,4466.966045,1.393907,NaN
1,ABB,2024-01-10 09:15:00,4935.70020,Buy,4819.371133,3.425688,NaN
2,ABB,2024-03-27 09:15:00,6279.35010,Buy,5888.662096,18.218708,NaN
3,ABB,2024-05-06 09:15:00,6940.35010,Buy,6619.356940,0.703577,NaN
4,ABB,2024-06-18 09:15:00,8867.00000,Buy,8460.521429,17.420153,NaN
...,...,...,...,...,...,...,...
470,ZYDUSLIFE,2023-12-07 09:15:00,657.95001,Buy,639.164111,0.532498,NaN
471,ZYDUSLIFE,2023-12-18 09:15:00,680.59998,Buy,648.748577,1.611368,NaN
472,ZYDUSLIFE,2024-01-24 09:15:00,752.40002,Buy,720.888368,0.772722,NaN
473,ZYDUSLIFE,2024-03-11 09:15:00,994.45001,Buy,954.862414,0.360444,NaN


In [114]:
all_cases_df.to_csv('ema-macd-actions.csv', index=False)

In [115]:
all_cases_df

,stock_name,time,close_price,action,ema9_max,macd-signal,ema9_min
0,ABB,2023-08-09 09:15:00,4520.95020,Buy,4466.966045,1.393907,NaN
1,ABB,2024-01-10 09:15:00,4935.70020,Buy,4819.371133,3.425688,NaN
2,ABB,2024-03-27 09:15:00,6279.35010,Buy,5888.662096,18.218708,NaN
3,ABB,2024-05-06 09:15:00,6940.35010,Buy,6619.356940,0.703577,NaN
4,ABB,2024-06-18 09:15:00,8867.00000,Buy,8460.521429,17.420153,NaN
...,...,...,...,...,...,...,...
470,ZYDUSLIFE,2023-12-07 09:15:00,657.95001,Buy,639.164111,0.532498,NaN
471,ZYDUSLIFE,2023-12-18 09:15:00,680.59998,Buy,648.748577,1.611368,NaN
472,ZYDUSLIFE,2024-01-24 09:15:00,752.40002,Buy,720.888368,0.772722,NaN
473,ZYDUSLIFE,2024-03-11 09:15:00,994.45001,Buy,954.862414,0.360444,NaN


In [116]:
target_date = datetime.strptime('2024-06-04', '%Y-%m-%d') #Election result date

In [117]:
def closest_to_target(group, target_date):
 return group.loc[(group['time'] - target_date).abs().idxmin()]

In [118]:
post_election_result_df = all_cases_df.groupby('stock_name').apply(closest_to_target, target_date).reset_index(drop=True)

C:\Users\91977\AppData\Local\Temp\ipykernel_7448\3036847206.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  post_election_result_df = all_cases_df.groupby('stock_name').apply(closest_to_target, target_date).reset_index(drop=True)


In [119]:
post_election_result_df

,stock_name,time,close_price,action,ema9_max,macd-signal,ema9_min
0,ABB,2024-06-18 09:15:00,8867.00000,Buy,8460.521429,17.420153,NaN
1,ADANIENSOL,2024-01-04 09:15:00,1179.80000,Buy,1095.291880,3.095250,NaN
2,ADANIENT,2024-02-26 09:15:00,3327.75000,Buy,3250.266302,1.158621,NaN
3,ADANIGREEN,2024-02-07 09:15:00,1866.00000,Buy,1716.780501,2.974487,NaN
4,ADANIPORTS,2024-02-26 09:15:00,1338.15000,Buy,1302.703589,0.598309,NaN
...,...,...,...,...,...,...,...
95,VBL,2024-05-15 09:15:00,1499.40000,Buy,1479.894303,0.687900,NaN
96,VEDL,2024-05-17 09:15:00,442.64999,Buy,423.426902,0.154180,NaN
97,WIPRO,2024-05-30 09:15:00,436.95001,Sell,NaN,-0.982879,453.40327
98,ZOMATO,2024-01-31 09:15:00,139.55000,Buy,135.603501,0.129539,NaN


In [120]:
#Getting close price at particular date 
close_price_post_budget = pd.DataFrame()
# for i in range(len(stocks)):
#   stock_history = tv.get_hist(symbol=stocks[i], exchange='NSE', interval=Interval.in_daily, n_bars=1)
#   close_price_post_budget = pd.concat([close_price_post_budget, stock_history], ignore_index=True)
#   print(stocks[i],i)

retries = 5
for i in range(len(stocks)):
  symbol = stocks[i]
  for attempt in range(retries):
    # result_df = process_stock(symbol)
    stock_history = tv.get_hist(symbol=stocks[i], exchange='NSE', interval=Interval.in_daily, n_bars=1)
    close_price_post_budget = pd.concat([close_price_post_budget, stock_history], ignore_index=True)
    print(stocks[i],i)        
    if stock_history is not None:
        close_price_post_budget = pd.concat([stock_history, close_price_post_budget], ignore_index=True)
        break
    else:
        print(f"Retrying {symbol}... Attempt {attempt + 1} of {retries}")
        time.sleep(1)  

ABB 0
ADANIENSOL 1
ADANIENT 2


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


ADANIGREEN 3
Retrying ADANIGREEN... Attempt 1 of 5
ADANIGREEN 3
ADANIPORTS 4
ADANIPOWER 5
ATGL 6
AMBUJACEM 7
APOLLOHOSP 8
ASIANPAINT 9
DMART 10


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


AXISBANK 11
Retrying AXISBANK... Attempt 1 of 5
AXISBANK 11


...

TECHM 90
Retrying TECHM... Attempt 1 of 5
TECHM 90
TITAN 91


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


TORNTPHARM 92
Retrying TORNTPHARM... Attempt 1 of 5
TORNTPHARM 92
TRENT 93
ULTRACEMCO 94
UNITDSPR 95
VBL 96
VEDL 97
WIPRO 98
ZOMATO 99


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


ZYDUSLIFE 100
Retrying ZYDUSLIFE... Attempt 1 of 5
ZYDUSLIFE 100


In [121]:
close_price_post_budget 

,symbol,open,high,low,close,volume
0,NSE:ZYDUSLIFE,1175.95,1194.00,1167.55,1188.30,406651.0
1,NSE:ZOMATO,221.00,223.97,216.95,219.82,19640616.0
2,NSE:WIPRO,496.00,505.95,493.30,505.35,6099647.0
3,NSE:VEDL,428.85,430.30,424.60,427.40,9709834.0
4,NSE:VBL,1610.00,1648.80,1601.00,1647.60,1711056.0
...,...,...,...,...,...,...
197,NSE:VBL,1610.00,1648.80,1601.00,1647.60,1711056.0
198,NSE:VEDL,428.85,430.30,424.60,427.40,9709834.0
199,NSE:WIPRO,496.00,505.95,493.30,505.35,6099647.0
200,NSE:ZOMATO,221.00,223.97,216.95,219.82,19640616.0


In [122]:
close_price_post_budget = close_price_post_budget.drop_duplicates(subset='symbol')

In [123]:
close_price_post_budget = close_price_post_budget.iloc[::-1].reset_index(drop=True)

In [124]:
close_price_post_budget

,symbol,open,high,low,close,volume
0,NSE:ABB,7450.00,7766.00,7410.05,7651.10,227320.0
1,NSE:ADANIENSOL,1030.20,1054.00,1025.05,1035.20,570562.0
2,NSE:ADANIENT,2960.70,3017.65,2945.10,2977.15,746965.0
3,NSE:ADANIGREEN,1710.00,1752.00,1701.00,1738.40,409055.0
4,NSE:ADANIPORTS,1471.00,1499.45,1466.85,1489.80,1340010.0
...,...,...,...,...,...,...
96,NSE:VBL,1610.00,1648.80,1601.00,1647.60,1711056.0
97,NSE:VEDL,428.85,430.30,424.60,427.40,9709834.0
98,NSE:WIPRO,496.00,505.95,493.30,505.35,6099647.0
99,NSE:ZOMATO,221.00,223.97,216.95,219.82,19640616.0


In [125]:
close_price_post_budget['symbol'] = close_price_post_budget['symbol'].str.replace('NSE:', '', regex=False)

In [138]:
post_election_result_df

,stock_name,time,close_price,action,ema9_max,macd-signal,ema9_min
0,ABB,2024-06-18 09:15:00,8867.00000,Buy,8460.521429,17.420153,NaN
1,ADANIENSOL,2024-01-04 09:15:00,1179.80000,Buy,1095.291880,3.095250,NaN
2,ADANIENT,2024-02-26 09:15:00,3327.75000,Buy,3250.266302,1.158621,NaN
3,ADANIGREEN,2024-02-07 09:15:00,1866.00000,Buy,1716.780501,2.974487,NaN
4,ADANIPORTS,2024-02-26 09:15:00,1338.15000,Buy,1302.703589,0.598309,NaN
...,...,...,...,...,...,...,...
95,VBL,2024-05-15 09:15:00,1499.40000,Buy,1479.894303,0.687900,NaN
96,VEDL,2024-05-17 09:15:00,442.64999,Buy,423.426902,0.154180,NaN
97,WIPRO,2024-05-30 09:15:00,436.95001,Sell,NaN,-0.982879,453.40327
98,ZOMATO,2024-01-31 09:15:00,139.55000,Buy,135.603501,0.129539,NaN


In [136]:
close_price_post_budget = close_price_post_budget[close_price_post_budget['symbol'] != 'SHREECEM']

In [139]:
close_price_post_budget

,symbol,open,high,low,close,volume
0,ABB,7450.00,7766.00,7410.05,7651.10,227320.0
1,ADANIENSOL,1030.20,1054.00,1025.05,1035.20,570562.0
2,ADANIENT,2960.70,3017.65,2945.10,2977.15,746965.0
3,ADANIGREEN,1710.00,1752.00,1701.00,1738.40,409055.0
4,ADANIPORTS,1471.00,1499.45,1466.85,1489.80,1340010.0
...,...,...,...,...,...,...
96,VBL,1610.00,1648.80,1601.00,1647.60,1711056.0
97,VEDL,428.85,430.30,424.60,427.40,9709834.0
98,WIPRO,496.00,505.95,493.30,505.35,6099647.0
99,ZOMATO,221.00,223.97,216.95,219.82,19640616.0


In [140]:
post_election_result_df = post_election_result_df.join(close_price_post_budget['close'])

In [141]:
post_election_result_df

,stock_name,time,close_price,action,ema9_max,macd-signal,ema9_min,close
0,ABB,2024-06-18 09:15:00,8867.00000,Buy,8460.521429,17.420153,NaN,7651.10
1,ADANIENSOL,2024-01-04 09:15:00,1179.80000,Buy,1095.291880,3.095250,NaN,1035.20
2,ADANIENT,2024-02-26 09:15:00,3327.75000,Buy,3250.266302,1.158621,NaN,2977.15
3,ADANIGREEN,2024-02-07 09:15:00,1866.00000,Buy,1716.780501,2.974487,NaN,1738.40
4,ADANIPORTS,2024-02-26 09:15:00,1338.15000,Buy,1302.703589,0.598309,NaN,1489.80
...,...,...,...,...,...,...,...,...
95,VBL,2024-05-15 09:15:00,1499.40000,Buy,1479.894303,0.687900,NaN,1429.00
96,VEDL,2024-05-17 09:15:00,442.64999,Buy,423.426902,0.154180,NaN,1647.60
97,WIPRO,2024-05-30 09:15:00,436.95001,Sell,NaN,-0.982879,453.40327,427.40
98,ZOMATO,2024-01-31 09:15:00,139.55000,Buy,135.603501,0.129539,NaN,505.35


In [142]:
post_election_result_df = post_election_result_df.rename(columns={'close_price': 'entry_price', 'time':'entry_time', 'close':'post_budget_price'})

In [153]:
post_election_result_df['return_percentage'] = 100 * ((post_election_result_df['post_budget_price'] - post_election_result_df['entry_price']) / post_election_result_df['entry_price'])

In [154]:
post_election_result_df

,stock_name,entry_time,entry_price,action,ema9_max,macd-signal,ema9_min,post_budget_price,return_percentage
0,ABB,2024-06-18 09:15:00,8867.00000,Buy,8460.521429,17.420153,NaN,7651.10,-13.712642
1,ADANIENSOL,2024-01-04 09:15:00,1179.80000,Buy,1095.291880,3.095250,NaN,1035.20,-12.256315
2,ADANIENT,2024-02-26 09:15:00,3327.75000,Buy,3250.266302,1.158621,NaN,2977.15,-10.535647
3,ADANIGREEN,2024-02-07 09:15:00,1866.00000,Buy,1716.780501,2.974487,NaN,1738.40,-6.838156
4,ADANIPORTS,2024-02-26 09:15:00,1338.15000,Buy,1302.703589,0.598309,NaN,1489.80,11.332810
...,...,...,...,...,...,...,...,...,...
95,VBL,2024-05-15 09:15:00,1499.40000,Buy,1479.894303,0.687900,NaN,1429.00,-4.695211
96,VEDL,2024-05-17 09:15:00,442.64999,Buy,423.426902,0.154180,NaN,1647.60,272.212818
97,WIPRO,2024-05-30 09:15:00,436.95001,Sell,NaN,-0.982879,453.40327,427.40,-2.185607
98,ZOMATO,2024-01-31 09:15:00,139.55000,Buy,135.603501,0.129539,NaN,505.35,262.128269


In [156]:
post_election_result_df.to_csv('post_election_result_df.csv', index=False)